# Matching 3D coordinates, with transformations

* <https://adventofcode.com/2021/day/19>



In [1]:
from __future__ import annotations
from collections import deque
from dataclasses import dataclass
from functools import cached_property
from itertools import chain, combinations, permutations
from typing import Final, Optional
import math

import numpy as np
from scipy.spatial.distance import pdist
from scipy.sparse import lil_matrix


MIN_BEACONS_IN_COMMON = 12
MIN_COMMON_DISTANCES = math.comb(MIN_BEACONS_IN_COMMON, 2)


# Generate the 24 unique 3D rotation matrices
def _rotations() -> np.array:
    # the identity transformation matrix, [1 0 0 0] [0 1 0 0] [0 0 1 0] [0 0 0 1]
    eye = np.identity(4, dtype=np.int8)
    # all permutations of 0, 1 and 2, padded with 3 to keep the bottom eye row in place.
    # used to re-arrange the rows of the eye matrix
    rows = np.pad(
        np.array(list(permutations(range(3)))), ((0, 0), (0, 1)), constant_values=3
    )
    # the product of (-1, 1), times 3, with a 1 added to the end; these are the
    # signs for each row of the rotation matrix.
    signs = np.pad(
        np.array([-1, 1])[
            np.stack(np.meshgrid(*([np.arange(2)] * 3)), axis=-1).reshape(-1, 3)
        ],
        ((0, 0), (0, 1)),
        constant_values=1,
    )
    # produce the product of signs and rows.
    signs, rows = np.repeat(signs, rows.shape[0], axis=0), np.tile(
        rows, (signs.shape[0], 1)
    )
    # lower half of the transformation matrix, used to calculate permutation parity
    # see https://en.wikipedia.org/wiki/Parity_of_a_permutation
    tx, ty = np.tril_indices(3, -1)
    rowsparity = np.prod(np.sign(rows[:, tx] - rows[:, ty]), axis=-1)
    signsparity = np.prod(signs[:, :3], axis=1)
    # all signs and all permutations with the same parity
    signs, rows = signs[rowsparity == signsparity], rows[rowsparity == signsparity]
    count = signs.shape[0]
    # alter eye with the signs combo (cols 0-3), then permute the rows (cols 4-7)
    return (signs[:, :, None] * np.tile(eye, (count, 1)).reshape(-1, 4, 4))[
        np.arange(count)[:, None], rows
    ]


ROTATIONS: Final[np.array] = _rotations()
NOR: int = ROTATIONS.shape[0]


def apply_transform(m: np.array, t: np.array) -> np.array:
    aug = np.concatenate((m, np.ones((*m.shape[:-1], 1), dtype=m.dtype)), axis=-1)
    return (aug @ t)[..., :-1]


@dataclass
class Scanner:
    beacons: np.array
    position: np.array = np.zeros(3, dtype=np.int16)

    @classmethod
    def from_lines(cls, lines: list[str]) -> Scanner:
        return Scanner(np.genfromtxt(lines, delimiter=",", dtype=np.int16))

    @cached_property
    def distances(self) -> dict[np.float64, tuple[int, int]]:
        """Map from distance to pair of indices of beacons"""
        map, combos = {}, combinations(range(self.beacons.shape[0]), 2)
        for dist, pair in zip(pdist(self.beacons), combos):
            try:
                map[dist].add(pair)
            except KeyError:
                map[dist] = {pair}
        return map

    @cached_property
    def orientations(self) -> np.array:
        # pad with 1s, compute the transformations, then un-pad.
        return apply_transform(self.beacons, ROTATIONS)
    
    def __and__(self, other: Scanner) -> Optional[Scanner]:
        """Check for scanner overlap

        Returns new Scanner at correct rotation with position updated, relative to
        other.

        """
        # how many distances are the same? If enough match, there is overlap
        intersection = self.distances.keys() & other.distances.keys()
        if sum(len(self.distances[d]) for d in intersection) < MIN_COMMON_DISTANCES:
            return None
        # track some of the sizes involved, number of other and self beacons and
        # orientations (reposititioned)
        nob = other.beacons.shape[0]
        nsb = self.beacons.shape[0]
        nsr = nsb * NOR
        # pick one of the beacons from other that we know has distances in common
        distance = next(iter(intersection))
        (other_a, _), *_ = other.distances[distance]
        reference = other.beacons[other_a]
        own_pairs = self.distances[distance]
        # try both ends of the matching pair in self; we don't know what side
        # matches with the reference beacon.
        for i in chain(*own_pairs):
            offsets = reference - self.orientations[:, i]
            repositioned = self.orientations + offsets[:, None, :]
            # find unique vectors, and their inverse index, used to quantify
            # how many vectors in a repositioned oriention fit.
            values, ix = np.unique(
                np.vstack((other.beacons, repositioned.reshape(-1, 3))),
                axis=0,
                return_inverse=True,
            )
            if values.shape[0] > nsr + nob - 12:
                continue  # not enough overlap between target beacons and repositioned

            # find the matching orientation intersecting the beacons and repositioned
            # matrix as sets; we count the unique values and create boolean matrices
            # mapping vector values to their index in the beacons and repositioned
            # matrices, then taking the dot product of these two mappings.
            ix_beacons, ix_repos = ix[:nob], ix[nob:]
            obvmembers = lil_matrix((nob, values.shape[0]), dtype=bool)
            obvmembers[np.arange(nob), ix_beacons] = True
            rvmembers = lil_matrix((nsr, values.shape[0]), dtype=bool)
            rvmembers[np.arange(nsr), ix_repos] = True
            matches = (obvmembers.tocsr() @ rvmembers.tocsr().T).T.sum(axis=-1)
            counts = matches.reshape(-1, nsb).sum(axis=-1)
            if not np.any(counts >= MIN_BEACONS_IN_COMMON):
                continue

            # orientation determined, get the corrected beacon positions
            orientation = np.argmax(counts)
            new_beacons = repositioned[orientation]
            new_pos = reference - self.orientations[orientation][i]
            return Scanner(new_beacons, new_pos)


@dataclass
class BeaconMap:
    scanners: list[Scanner]

    @classmethod
    def from_text(cls, text: str) -> BeaconMap:
        scanners = [
            Scanner.from_lines(sc.splitlines()[1:]) for sc in text.split("\n\n")
        ]
        return cls(scanners)

    @cached_property
    def positioned_scanners(self):
        to_position = deque(self.scanners)
        positioned = [to_position.popleft()]
        while to_position:
            scanner = to_position.popleft()
            for other in positioned:
                if placed := scanner & other:
                    positioned.append(placed)
                    break
            else:
                to_position.append(scanner)
        return positioned
    
    @cached_property
    def positioned_beacons(self):
        return {pos for s in self.positioned_scanners for pos in zip(*s.beacons.T)}


test_map = BeaconMap.from_text(
    """\
--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14
"""
)

assert len(test_map.positioned_beacons) == 79

In [2]:
import aocd

beacon_map = BeaconMap.from_text(aocd.get_data(day=19, year=2021))
print("Part 1:", len(beacon_map.positioned_beacons))

Part 1: 400


In [3]:
def max_distance(scanners: list[Scanner]) -> int:
    return int(pdist(np.array([s.position for s in scanners]), 'cityblock').max())


assert max_distance(test_map.positioned_scanners) == 3621

In [4]:
print("Part 2:", max_distance(beacon_map.positioned_scanners))

Part 2: 12168
